# Pretrained Models and Transfer Learning

By using pre-trained models and transfer learning, the hard work of training an AI model from zero can be bypassed, making it easier and quicker to get the job done. By leveraging the knowledge a model distills from a large dataset, we can reduce the amount of training needed to get a performant model.

**Examples:**
* If we wanted to create a plant identification app for mobile devices, we might use [MobileNetV3](https://paperswithcode.com/method/mobilenetv3) and train it on a dataset containing photos of different plant species.
* If we wanted to create a social networking spam classifier, we might use BERT and train it on a dataset containing samples of spam and not-spam text.


**Transfer learning:** The process where knowledge from a pre-trained model is applied to a new, but related task.

<img src="img/img_01.png">
<br>
<img src="img/img_02.png">
<br>

**Example of creating a plant classification model**:
<img src="img/img_03.png">

**Example of creating a spam classification model**:
<img src="img/img_04.png">